In [1]:
import pickle 
import pandas as pd
from sqlalchemy import create_engine, text
def insert_pred_to_sql():
    # 读取数据库连接字符串
    with open('./datasets/sql_token.pkl', 'rb') as f:
        DB_URI = pickle.load(f)

    # 创建数据库引擎
    engine = create_engine(DB_URI)

    # SQL 查询语句
    sql = text("""
        SELECT fund_code, forecast_date, pre_data, model_version, create_time, update_time
        FROM b_fund_forecast_new
        WHERE fund_code IN :codes
        ORDER BY forecast_date
    """)

    # 执行查询，传入参数（注意 tuple 中只有一个元素时加逗号）
    df = pd.read_sql_query(
        sql.bindparams(codes=tuple(['005626'])),  # 或 codes=('005626',)
        engine
    )
    
    df.to_sql('my_table', engine, if_exists='replace', index=False)
    return True

In [2]:
# import numpy as np 
# for i in range(len(df)):
#     pred_str = df['pre_data'][i]
#     str_start = pred_str.find('[') + 1
#     str_end = pred_str.find(']') 
#     pred = pred_str[str_start:str_end].split(', ')
#     pred = np.array(pred, dtype=np.float32)
#     # print(pred.shape)

In [3]:
from datetime import datetime, timedelta

def get_start_date(end_date: str, window_size: int) -> str:
    """
    给定结束日期和历史窗口长度，返回窗口开始日期（字符串格式）。

    参数：
    - end_date (str): 结束日期，格式 'YYYY-MM-DD'
    - window_size (int): 历史窗口长度（天数）

    返回：
    - start_date (str): 开始日期，格式 'YYYY-MM-DD'
    """
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    start_dt = end_dt - timedelta(days=window_size)
    return start_dt.strftime("%Y-%m-%d")

In [7]:
from data_provider.generate_financial import process_date_columns, query_fund_data
from data_provider.get_financial import get_group_idx
from utils.exp_config import get_config
import pickle 
from sqlalchemy import create_engine, text
import numpy as np

config = get_config('FinancialConfig')

# 读取数据库连接字符串
with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

# 创建数据库引擎
engine = create_engine(DB_URI)
now_fund_code = get_group_idx(27)
end_date = '2025-4-15'
all_history_input = []
for i in range(len(now_fund_code)):
    start_date = get_start_date(end_date, window_size=64)
    df = query_fund_data(now_fund_code[i], start_date, end_date)
    df = process_date_columns(df)
    all_history_input.append(df)

df = all_history_input

In [8]:
data = np.stack(df, axis=0)
data = data.transpose(1, 0, 2)
x = data[:, :, :]
x.shape

(46, 68, 8)

In [5]:
# import pandas as pd
# from datetime import datetime

# def convert_array_to_df(array):
#     """
#     将数组转换为包含标准日期列的 DataFrame
#     """
#     df = pd.DataFrame(array, columns=[
#         'fund_code', 'year', 'month', 'day', 'weekday',
#         'value1', 'value2', 'value3'
#     ])
#     df['year'] = df['year'].astype(int)
#     df['month'] = df['month'].astype(int)
#     df['day'] = df['day'].astype(int)
#     df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
#     return df

# def find_missing_dates_from_array_list(array_list, start_date, end_date, freq='B'):
#     """
#     参数：
#     - array_list: List[np.ndarray]，每个元素是一个基金的历史数据数组
#     - start_date, end_date: 'YYYY-MM-DD'
#     - freq: 'B' 表示工作日（默认）

#     返回：
#     - dict: {fund_code: [缺失的日期列表]}
#     """
#     full_range = pd.date_range(start=start_date, end=end_date, freq=freq)
#     missing_map = {}

#     for arr in array_list:
#         df = convert_array_to_df(arr)
#         if df.empty:
#             continue

#         fund_code = df.iloc[0]['fund_code']
#         date_series = df['date']
#         missing = full_range.difference(date_series)

#         if not missing.empty:
#             missing_map[fund_code] = missing.strftime('%Y-%m-%d').tolist()

#     return missing_map

# missing_map = find_missing_dates_from_array_list(df, start_date, end_date)

# for code, dates in missing_map.items():
#     print(f"基金 {code} 缺失日期：{dates}")

In [12]:
data = np.stack(df, axis=0)
data = data.transpose(1, 0, 2)
x = data[:, :, :]
x.shape

(46, 68, 8)

In [17]:
config.num_layers

2

In [12]:
import torch

state = torch.load('checkpoints/ours/Model_ours_Dataset_financial_Multi_round_0.pt')

if isinstance(state, dict):
    # 一般是 state_dict
    print("Keys in state_dict:")
    for key, value in state.items():
        print(f"{key} : {value.shape}")
else:
    # 如果是完整模型
    print(state)  # 会自动打印结构

Keys in state_dict:
model.projection.weight : torch.Size([56, 3])
model.projection.bias : torch.Size([56])
model.position_embedding.pos_encoding.embedding.weight : torch.Size([17, 56])
model.fund_embedding.weight : torch.Size([999999, 56])
model.predict_linear.weight : torch.Size([26, 16])
model.predict_linear.bias : torch.Size([26])
model.encoder.layers.0.0.weight : torch.Size([56])
model.encoder.layers.0.1.att.in_proj_weight : torch.Size([168, 56])
model.encoder.layers.0.1.att.in_proj_bias : torch.Size([168])
model.encoder.layers.0.1.att.out_proj.weight : torch.Size([56, 56])
model.encoder.layers.0.1.att.out_proj.bias : torch.Size([56])
model.encoder.layers.0.2.weight : torch.Size([56])
model.encoder.layers.0.3.net.0.weight : torch.Size([112, 56])
model.encoder.layers.0.3.net.0.bias : torch.Size([112])
model.encoder.layers.0.3.net.2.weight : torch.Size([56, 112])
model.encoder.layers.0.3.net.2.bias : torch.Size([56])
model.encoder.layers.1.0.weight : torch.Size([56])
model.encoder.la

/tmp/ipykernel_411756/2551740208.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load('checkpoints/ours/Model_ours_Dataset_financial_Multi_round_0.pt')


In [14]:
from modules.backbone import Backbone
import torch 
model = Backbone(3, config).load_state_dict(torch.load('checkpoints/ours/Model_ours_Dataset_financial_Multi_round_0.pt'))

/tmp/ipykernel_411756/804212040.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = Backbone(3, config).load_state_dict(torch.load('checkpoints/ours/Model_ours_Datas

RuntimeError: Error(s) in loading state_dict for Backbone:
	Missing key(s) in state_dict: "projection.weight", "projection.bias", "position_embedding.pos_encoding.embedding.weight", "fund_embedding.weight", "predict_linear.weight", "predict_linear.bias", "encoder.layers.0.0.weight", "encoder.layers.0.1.att.in_proj_weight", "encoder.layers.0.1.att.in_proj_bias", "encoder.layers.0.1.att.out_proj.weight", "encoder.layers.0.1.att.out_proj.bias", "encoder.layers.0.2.weight", "encoder.layers.0.3.net.0.weight", "encoder.layers.0.3.net.0.bias", "encoder.layers.0.3.net.2.weight", "encoder.layers.0.3.net.2.bias", "encoder.layers.1.0.weight", "encoder.layers.1.1.att.in_proj_weight", "encoder.layers.1.1.att.in_proj_bias", "encoder.layers.1.1.att.out_proj.weight", "encoder.layers.1.1.att.out_proj.bias", "encoder.layers.1.2.weight", "encoder.layers.1.3.net.0.weight", "encoder.layers.1.3.net.0.bias", "encoder.layers.1.3.net.2.weight", "encoder.layers.1.3.net.2.bias", "encoder.norm.weight", "encoder2.

In [10]:
#%%
df = np.array(all_history_input)
df.shape
#%%
data = np.stack(df, axis=0)
data = data.transpose(1, 0, 2)
x = data[:, :, :]